In [16]:
# Model Application - Map YAML to current web app JSON input format.

import json
import yaml

with open(r"C:\py\hec_meta_extract\example\input\json\ras_simulation.json", 'r') as f:
    ras_sim_template_json = json.load(f)

# type(ras_model_template_json)
# print(ras_model_template_json.keys())

# keys to drop
drop_keys = ['_id', 'model_software', 'parameters', 'linked_resources', 'type', 'output_files']

for key in drop_keys:
    del ras_sim_template_json[key]

In [18]:
# Open ras prj yaml to map values to json
p_yaml_fn = r'C:\py\hec_meta_extract\output\ras\Amite_2022\Amite_2022.p07.yml'
with open(p_yaml_fn, "r") as stream:
    try:
        p_yaml = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [13]:
# Format Simulation Date to match web app format
from datetime import datetime 
dt_raw = p_yaml['Simulation Date'].split(",")
sDate_dt = datetime.strptime(dt_raw[0], '%d%b%Y')
sDate_str = datetime.strftime(sDate_dt, '%Y-%m-%d')
eDate_dt = datetime.strptime(dt_raw[2], '%d%b%Y')
eDate_str = datetime.strftime(eDate_dt, '%Y-%m-%d')
temporal_extent = [sDate_str, eDate_str]



In [52]:
import os
p_num = p_yaml_fn.split(".")[-2][1:]
g_num = p_yaml['Geom File'][1:]
u_num = p_yaml['Flow File'][1:]

# prj_name = p_yaml_fn.split(".")[-3]
prj_dir, prj_file_tail = os.path.split(p_yaml_fn)
prj_name = prj_file_tail.split(".")[0]
# prj_name
b_file = 'f{prj_name}.b{p_num}'
b_file


'f{prj_name}.b{p_num}'

In [31]:
# Get Unique input DSS files
# p_yaml['DSS Input Files']

dss_input_file_list = [el[0] for el in p_yaml['DSS Input Files']]
# dss_input_file_list
list_unique = list(set(dss_input_file_list))
list_unique

['Met BC=Wind Velocity Y|Gridded DSS Filename=.\\BC_Extended\\ADCIRC_WIND_6\\Isaac_Y.dss',
 'DSS File=.\\BC_Extended\\ADCIRC_WL_Isaac.dss',
 'DSS File=.\\BC_Extended\\AORC_Data\\Isaac___Grid___AORC.dss',
 'Met BC=Precipitation|Gridded DSS Filename=.\\BC_Extended\\AORC_Data\\Gridded_Rainfall\\Isaac_AORC.dss',
 'Met BC=Wind Velocity X|Gridded DSS Filename=.\\BC_Extended\\ADCIRC_WIND_6\\Isaac_X.dss',
 'Observed Time Series=Stage|TS DSS Filename=.\\BC_Extended\\PortVincent_AmiteGage_15min.dss']

In [60]:
input_files = []
for i in list_unique:
    j = i.split("=")
    # print (j)
    # print ('\n')
    if len(j) == 3:
        input_files.append(
            {
                "title": j[0]+" "+j[1],
                "location": j[-1],
                "description": j[0]+" "+j[1],
                "source_dataset": None
            }
        )
    else:
        input_files.append(
            {
                "title": j[0],
                "location": j[-1],
                "description": j[0],
                "source_dataset": None
            }
        )
input_files.extend(
    [{
        "title": "Terrain",
        "location": p_yaml['terrain'],
        "description": "Terrain used by model",
        "source_dataset": None
    },
    {
        "title": "b file",
        "location": 'f{prj_name}.b{p_num}',
        "description": "RAS master input text file",
        "source_dataset": None
    },
    {
        "title": "g file",
        "location": f"{prj_name}.{p_yaml['Geom File']}",
        "description": "RAS geometry file",
        "source_dataset": None
    },
    {
        "title": "prj file",
        "location": f'{prj_name}.prj',
        "description": "RAS project file which links projects with plans, geometry, and flow files",
        "source_dataset": None
        },
    {
        "title": "c file",
        "location": f"{prj_name}.c{g_num}",
        "description": "Binary Geometry file from Geom Prep",
        "source_dataset": None
    },
        {
            "title": "x file",
            "location": f"{prj_name}.x{g_num}",
            "description": "Geometry master input text file",
            "source_dataset": None
        },
        {
            "title": "p file",
            "location": f"{prj_name}.p{p_num}",
            "description": "Model plan data",
            "source_dataset": None
        },
        {
            "title": "u file",
            "location": f"{prj_name}.u{u_num}",
            "description": "unsteady flow file",
            "source_dataset": None
        },
        {
            "title": "u hdf file",
            "location": f"{prj_name}.u{u_num}.hdf",
            "description": "unsteady flow file in HDF format",
            "source_dataset": None
        }]
)
input_files

[{'title': 'Met BC Wind Velocity Y|Gridded DSS Filename',
  'location': '.\\BC_Extended\\ADCIRC_WIND_6\\Isaac_Y.dss',
  'description': 'Met BC Wind Velocity Y|Gridded DSS Filename',
  'source_dataset': None},
 {'title': 'DSS File',
  'location': '.\\BC_Extended\\ADCIRC_WL_Isaac.dss',
  'description': 'DSS File',
  'source_dataset': None},
 {'title': 'DSS File',
  'location': '.\\BC_Extended\\AORC_Data\\Isaac___Grid___AORC.dss',
  'description': 'DSS File',
  'source_dataset': None},
 {'title': 'Met BC Precipitation|Gridded DSS Filename',
  'location': '.\\BC_Extended\\AORC_Data\\Gridded_Rainfall\\Isaac_AORC.dss',
  'description': 'Met BC Precipitation|Gridded DSS Filename',
  'source_dataset': None},
 {'title': 'Met BC Wind Velocity X|Gridded DSS Filename',
  'location': '.\\BC_Extended\\ADCIRC_WIND_6\\Isaac_X.dss',
  'description': 'Met BC Wind Velocity X|Gridded DSS Filename',
  'source_dataset': None},
 {'title': 'Observed Time Series Stage|TS DSS Filename',
  'location': '.\\BC_Ext

In [62]:
# Map

# ras_model_template_json['title'] = prj_yaml['Proj Title']
# ras_model_template_json['description'] = prj_yaml['Description']
# ras_model_template_json['purpose'] = prj_yaml['Description']
# ras_model_template_json['simulations'] = prj_yaml['Plans']
# ras_model_template_json['grid']['coordinate_system'] = prj_yaml['coordinate_system']
# ras_model_template_json['spatial_extent'][0] = prj_yaml['spatial_extent']
# ras_model_template_json['common_files_details'][0] = {
#     'source_dataset': None,
#     'description': 'RAS project file which links projects with plans, geometry, and flow files',
#     'location': prj_yaml['Project File'],
#     'title': 'prj file'
# }

ras_sim_template_json['title'] = p_yaml['Plan Title']
ras_sim_template_json['description'] = p_yaml['Description']
ras_sim_template_json['temporal_extent'] = temporal_extent
ras_sim_template_json['temporal_resolution'] = p_yaml['Computation Interval']
ras_sim_template_json['output_files'] = [
    {
            "title": "output dss file",
            "location": p_yaml['DSS Output File'],
            "description": "output model data in dss",
            "source_dataset": None
        },
        {
            "title": "p hdf file",
            "location": f"{prj_name}.p{p_num}.hdf",
            "description": "result output in HDF format",
            "source_dataset": None
        }
]
ras_sim_template_json['input_files'] = input_files




In [64]:
# output mapped json
with open("simTest.json", "w") as outfile:
    json.dump(ras_sim_template_json, outfile)